In [8]:
import numpy as np
import emcee
import pandas as pd
import corner
from scipy.optimize import fsolve

import time
import multiprocessing
from itertools import product
from functools import partial

%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

In [3]:
def lnlikelihood(theta, f, t, f_err):
    a, a_prime, t_0, t_b, alpha_r, alpha_d, s, sig_0 = theta

    pre_exp = np.logical_not(t > t_0)
    model = np.empty_like(f)
    model[pre_exp] = a
    
    time_term = (t[~pre_exp] - t_0)/t_b
    model[~pre_exp] = a_prime * (time_term)**alpha_r * (1 + (time_term)**(s*alpha_d))**(-2/s)
    
    ln_l = np.sum(np.log(1. / np.sqrt(2*np.pi * (sig_0**2 + f_err**2))) - ((f - model)**2 / (2 * (sig_0**2 + f_err**2))))
    return ln_l

#Define priors on parameters  
def lnprior(theta):
    a, a_prime, t_0, t_b, alpha_r, alpha_d, s, sig_0 = theta
    if (-100 < t_0 < 100 and 0 < alpha_r < 100 and 
        0 < alpha_d < 100 and 0 < sig_0 < 10000 and 
        -1000 < a < 1000 and  0 < t_b < 1000 and 
        0 < s < 1000 and 0 < a_prime < 1e6):
        return 0.0
    return -np.inf

def lnposterior(theta, f, t, f_err):
    lnp = lnprior(theta)
    lnl = lnlikelihood(theta, f, t, f_err)
    if not np.isfinite(lnl):
        return -np.inf
    if not np.isfinite(lnp):
        return -np.inf
    return lnl + lnp 

In [21]:
def sn_lc_mcmc(sigma_sys, sn_lc_obj, loop_num):
    
    sn_lc_obj.calc_noisy_lc(sigma_sys=sigma_sys)
        
    t_rest = sn_lc_obj.t_obs_/(1 + sn_lc_obj.z_)
    f_data = sn_lc_obj.cnts_
    f_unc_data = sn_lc_obj.cnts_unc_

    #initial guess on parameters
    guess_0 = [0, 1e4, 0, 20, 2, 2, 1, 10]

    #number of walkers
    nwalkers = 100
    nfac = [1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2]
    ndim = len(guess_0)

    #initial position of walkers
    pos = [guess_0 + nfac * np.random.randn(ndim) for i in range(nwalkers)]

    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnposterior, 
                                    args=(f_data, t_rest, f_unc_data))
    nsamples = 500
    foo = sampler.run_mcmc(pos, nsamples)

    # intermediate file to write out data
    filename = "noise{}_loop{}.h5".format(sigma_sys, loop_num)
    backend = emcee.backends.HDFBackend(filename)
    backend.reset(nwalkers, ndim)
    
    # run second burn in
    burn_max_post = sampler.chain[:,-1,:][np.argmax(sampler.lnprobability[-1,:])]
    pos = [burn_max_post + nfac * np.random.randn(ndim) for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnposterior, 
                                    args=(f_data, t_rest, f_unc_data), 
                                    backend=backend)
    nsamples = 1000
    foo = sampler.run_mcmc(pos, nsamples)
    
    return sampler

In [19]:
sim_dict = {}

for sim_num in range(3):
    sn = SimSnIa()
    sn.draw_dist_in_volume(d_max=600)
    sn.draw_alpha_r()
    sn.draw_rise_time()
    sn.draw_smoothing_parameter()
    sn.draw_mb_deltam15()
    sn.calc_alpha_d()
    sn.calc_a_prime()

    t_obs = np.arange(-40, 35, 1, dtype=float) + np.random.uniform(-0.25/24,0.25/24,size=75)

    sn.calc_ft(t_obs)
    # sn.calc_noisy_lc(sigma_sys=8)

    pool = multiprocessing.Pool(4)
    sampler1, sampler2, sampler3, sampler4 = pool.map(partial(sn_lc_mcmc, sn_lc_obj=sn, loop_num=sim_num), 
                                                      [3,5,50,316])
    
    loop_dict = {'dist': sn.dist_,
                 'alpha_r': sn.alpha_r_,
                 't_p': sn.t_p_,
                 't_b': sn.t_b_, 
                 's': sn.s_,
                 'alpha_d': sn.alpha_d_,
                 'a_prime': sn.a_prime_,
                 't_exp': sn.t_exp_}
    
    sim_dict['sim_num{}'.format(sim_num)] = loop_dict


/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in power
  if __name__ == '__main__':
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in power
  if __name__ == '__main__':
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in square
  # This is added back by InteractiveShellApp.init_path()
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in power
  if __name__ == '__main__':
/Users/adamamiller/miniconda3/envs/emcee3/lib/python3.7/site-packages/ipykernel_launcher

In [20]:
sim_dict

{'': {'dist': 596.0987350841237,
  'alpha_r': 1.1005289552316893,
  't_p': 17.975542796819003,
  't_b': 22.23572193673273,
  's': 2.263898279523644,
  'alpha_d': 1.9733147813153846,
  'a_prime': 313.1022082619749,
  't_exp': 0}}

In [22]:
def delta_m15_root(alpha_d, t_p=18, alpha_r=2, s=1, dm15=1.1):
    '''Root solver for alpha_d based on Delta m15
    
    Using Eqn. 4 from Zheng & Filippenko (2017), ApJL, 838, 4, it is 
    possible to calculate the ratio of flux from a SN Ia at t = t_peak
    and t = t_peak + 15. If t_p, alpha_r, and s are known, then the 
    ratio of flux should equal Delta m15. The scipy.optimize root 
    finder fsolve is used to solve for the value of alpha_d.
    
    Parameters
    ----------
    alpha_d : float
        Power-law index for the late-time decline of the SN
    
    t_p : float, optional (default=18)
        Time to peak of the SN light curve

    alpha_r : float, optional (default=2)
        Power-law index for initial rise of the SN light curve 

    s : float, optional (default=1)
        Smoothing parameter for the light curve

    dm15 : float, optional (default=1.1)
        Delta m15
    
    Returns
    -------
    alpha_d_root
        The value of alpha_d that results in a SN light curve
        with a 15 day decline rate = Delta m15
    '''
    
    t_b = t_p/((-alpha_r/2)/(alpha_r/2 - alpha_d))**(1/(s*(alpha_d)))
    
    Ltp = (t_p/t_b)**alpha_r * (1 + (t_p/t_b)**(s*alpha_d))**(-2/s)
    Ltp_15 = ((t_p + 15)/t_b)**alpha_r * (1 + ((t_p + 15)/t_b)**(s*alpha_d))**(-2/s)
    
    return 2.5*np.log10(Ltp/Ltp_15) - dm15


class SimSnIa():
    
    def __init__(self):
        '''initialize the simulated SN'''
    
    def draw_dist_in_volume(self, d_max=100, H_0=72):
        '''simulate SN at a random distance within a fixed volume
        
        Parameters
        ----------
        d_max : int, optional (default=100)
            Maximum distance for the simulated SNe, units in Mpc
        
        H_0 : float, optional (default=72)
            Value of the Hubble constant (in km/s/Mpc) used to convert the 
        distance to the SN to a redshift, z.
        
        Attributes
        ----------
        dist_ : float
            Distance to the SN in Mpc
        
        z_ : float
            Redshift to the SN
        
        mu_ : float
            distance modulus to the SN            
        '''
        
        self.dist_ = np.random.uniform()**(1/3)*d_max
        self.z_ = H_0*self.dist_/2.997942e5
        self.mu_ = 5*np.log10(self.dist_) + 25
    
    def draw_alpha_r(self, alpha_low=1, alpha_high=2.5):
        '''draw random value for early rise power-law index
        
        Select a random value from a flat distribution between 
        alpha_low and alpha_high to determine the power-law index
        for the initial rise of the SN light curve.
        
        Parameters
        ----------
        alpha_low : float, optional (default=1)
            Minimum value for the power-law index of the early rise
        
        alpha_high : float, optional (default=2.5)
            Maximum value for the power-law index of the early rise
        
        Attributes
        ----------
        alpha_r_ : float
            Power-law index for initial rise of the SN light curve          
        '''
        
        self.alpha_r_ = np.random.uniform(alpha_low, alpha_high)
    
    def draw_rise_time(self, mu_rise=18, sig_rise=1):
        '''draw random value for the light curve rise time
        
        Select a random value from a gaussian distribution with 
        mean, mu_rise (default=18), and standard deviation, 
        sig_rise (default=1). The defaults are selected based on the 
        results from Ganeshalingam et al. 2011, MNRAS, 416, 2607 
        which found that the rise time for SNe Ia can be described 
        as ~ N(18.03, 0.0576).
        
        Parameters
        ----------
        mu_rise : float, optional (default=18)
            Mean value for the rise time of SN Ia
        
        sig_rise : float, optional (default=1)
            Standard deviation of the rise time distribution for 
            SNe Ia
        
        Attributes
        ----------
        t_p_ : float
            Time for the light curve to reach peak brightness          
        '''
        
        self.t_p_ = np.random.normal(mu_rise, sig_rise)
        
    def draw_smoothing_parameter(self, mu_s=2, sig_s=0.5):
        '''draw random value for the smoothing parameter
        
        Select a random value from a truncated gaussian distribution 
        with mean, mu_s (default=2), and standard deviation, 
        sig_s (default=0.5). This parameter is not physical, and 
        is largely degenerate with alpha_decline. It is drawn from 
        a guassian distribution while alpha_decline is selected to 
        ensure a physical value of delta m15.
        
        Parameters
        ----------
        mu_s : float, optional (default=2)
            Mean value for the smoothing parameter
        
        sig_s : float, optional (default=0.5)
            Standard deviation of the smoothing parameter
        
        Attributes
        ----------
        s_ : float
            Smoothing parameter for the light curve          
        '''
        s = -1
        while s < 0:
            s = np.random.normal(mu_s, sig_s)
        
        self.s_ = s
    
    def draw_mb_deltam15(self, mu_m_b=-19.3, mu_dm15=1.1,
                         cov=[[0.04,0.042],[0.042,0.09]]):
        '''Draw random M_b and Delta m15 values
        
        Draw from multivariate gaussian to get M_b and Delta m15
        using mu_m_b (default=-19.3), mu_dm15 (default=1.1), and 
        covariance matrix, cov (default=[[0.04,0.042],[0.042,0.09]]).
        The default distributions for the multivariate gaussian are 
        generated from the population values from the SNe studied in 
        Prieto et al. 2006, ApJ, 647, 501.
        
        Parameters
        ----------
        mu_m_b : float, optional (default=-19.3)
            Mean value for the distribution of B-band peak absolute 
            magnitude
        
        mu_dm15 : float, optional (default=1.1)
            Mean value for the distribution of Delta m15 
            
        cov : 2x2 array, optional (default=[[0.04,0.042],[0.042,0.09]])
        
        Attributes
        ----------
        M_b_ : float
            Rest-frame absolute magnitude in the B band at the 
            time of peak brightness
        dm15_ : float
            Delta m15 for the SN
        '''

        self.M_b_, self.dm15_ = np.random.multivariate_normal([mu_m_b,mu_dm15],cov)

    def calc_alpha_d(self, alpha_d_guess=2):
        '''Calculate the value of alpha_d based on Delta m15
        
        Parameters
        ----------
        alpha_d_guess : float, optional (default=2)
            Initial guess to solve for the root of the alpha_d eqn
        
        Attributes
        ----------
        alpha_d_ : float
            Power-law index for the late-time decline of the SN
        '''
        if not (hasattr(self, 't_p_') and hasattr(self, 'alpha_r_') and 
                hasattr(self, 's_') and hasattr(self, 'dm15_')):
            self.draw_alpha_r()
            self.draw_rise_time()
            self.draw_smoothing_parameter()
            self.draw_mb_deltam15()

        alpha_d = fsolve(delta_m15_root, alpha_d_guess, 
                         args=(self.t_p_, self.alpha_r_, 
                               self.s_, self.dm15_))

        self.alpha_d_ = float(alpha_d)

    def calc_a_prime(self):
        '''Calculate the value of Aprime
        
        Determine the normalization constant to generate a 
        SN light curve with peak flux equal to the luminosity
        associated with M_b.
        
        Attributes
        ----------
        t_b_ : float
            "break time" for the broken power-law model
        
        a_prime_ : float
            Amplitude for the SN light curve
        '''
        if not (hasattr(self, 'alpha_d_') and hasattr(self, 'mu_')):
            self.draw_dist_in_volume()
            self.calc_alpha_d()
            
        m_peak = self.M_b_ + self.mu_

        f_peak = 10**(0.4*(25-m_peak))
        
        t_b = self.t_p_/((-self.alpha_r_/2)/(self.alpha_r_/2 - self.alpha_d_))**(1/(self.s_*(self.alpha_d_)))
        
        model_peak = ((self.t_p_)/t_b)**self.alpha_r_ * (1 + ((self.t_p_)/t_b)**(self.s_*self.alpha_d_))**(-2/self.s_)

        a_prime = f_peak/model_peak
        
        self.t_b_ = t_b
        self.a_prime_ = a_prime
    
    def calc_ft(self, t_obs, t_exp=0):
        '''Calculate the model flux at input times t_obs
        
        Use Eqn. 4 of Zheng & Filippenko 2017 to determine the 
        flux from the SN at all input times t_obs.
        
        Parameters
        ----------
        t_obs : array-like of shape = [n_obs]
            Times at which to calculate the flux from the SN
        
        t_exp : float, optional (default=0)
            Time of explosion for the SN model
        
        Attributes
        ----------
        t_obs_ : array-like of shape = [n_obs]
            Times at which the SN flux is measured
        
        t_exp_ : float
            SN time of explosion
            
        model_flux : array-like of shape = [n_obs]
            The model flux at all times t_obs, assuming no noise 
            contributes to the signal from the SN
        '''
        if not hasattr(self, 'a_prime_'):
            self.calc_a_prime()
        
        pre_explosion = np.logical_not(t_obs > t_exp)
        
        model_flux = np.empty_like(t_obs)
        model_flux[pre_explosion] = 0
        
        t_rest = t_obs[~pre_explosion]/(1 + self.z_)
        model_flux[~pre_explosion] = self.a_prime_ * ((( - t_exp)/self.t_b_)**self.alpha_r_ * 
                                     (1 + ((t_obs[~pre_explosion] - t_exp)/self.t_b_)**(self.s_*self.alpha_d_))**(-2/self.s_))
        
        self.t_obs_ = t_obs
        self.t_exp_ = t_exp
        self.model_flux_ = model_flux

    def calc_noisy_lc(self, sigma_sys=20):
        '''Calculate SN light curve with systematic and statistical noise
        
        Parameters
        ----------
        sigma_sys : float, optional (default=20)
            Systematic noise term to noisify the light curve. Telescope 
            system is assumed to have a zero-point of 25, such that 
            m = 25 - 2.5*log10(flux). Thus, 
            sigma_sys(5-sigma limiting mag) = 10**(0.4*(25 - m_lim))/5. 
            Default corresponds to a limiting mag of 20.
        
        Attributes
        ----------
        cnts : array-like of shape = [n_obs]
            noisy flux from the SN light curve
        
        cnts_unc : array-like of shape = [n_obs]
            uncertainty on the noisy flux measurements
        '''
        if not hasattr(self, 'model_flux_'):
            self.calc_ft()

        cnts = np.zeros_like(self.t_obs_)
        cnts_unc = np.zeros_like(self.t_obs_)

        pre_explosion = np.logical_not(self.t_obs_ > self.t_exp_)
        cnts[pre_explosion] = np.random.normal(0, sigma_sys, size=sum(pre_explosion))
        cnts_unc[pre_explosion] = np.ones_like(self.t_obs_)[pre_explosion]*sigma_sys

        sn_flux = self.model_flux_[~pre_explosion]
        sn_with_random_noise = sn_flux + np.random.normal(np.zeros_like(sn_flux), np.sqrt(sn_flux))
        sn_with_random_plus_sys = sn_with_random_noise + np.random.normal(0, sigma_sys, size=len(sn_flux))

        # total uncertainty = systematic + Poisson
        sn_uncertainties = np.hypot(np.sqrt(np.maximum(sn_with_random_noise, 
                                                       np.zeros_like(sn_with_random_noise))), 
                                    sigma_sys)

        cnts[~pre_explosion] = sn_with_random_plus_sys
        cnts_unc[~pre_explosion] = sn_uncertainties

        self.cnts_ = cnts
        self.cnts_unc_ = cnts_unc